<a href="https://colab.research.google.com/github/harshnarang8/sc779-comp/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model file is for testing how the transformerencoder and transformerencoderlayer modules in pytorch work. The code has mostly been sourced from the transformer tutorial on the pytorch website linked in the doc given in the cs779 course.

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class TransformerModel(nn.Module):

  def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
    super(TransformerModel, self).__init__()
    from torch.nn import TransformerEncoder, TransformerEncoderLayer
    self.model_type = 'Transformer'
    self.pos_encoder = PositionalEncoding(ninp, dropout)
    encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
    self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
    self.encoder = nn.Embedding(ntoken, ninp)
    self.ninp = ninp
    self.decoder = nn.Linear(ninp, ntoken)

    self.init_weights()

  def generate_square_subsequent_mask(self, sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
  
  def init_weights(self):
    initrange = 0.1
    self.encoder.weight.data.uniform_(-initrange, initrange)
    self.decoder.bias.data.zero_()
    self.decoder.weight.data.uniform_(-initrange, initrange)
  
  def forward(self, src, src_mask):
    src = self.encoder(src)*math.sqrt(self.ninp)
    src = self.pos_encoder(src)
    output = self.transformer_encoder(src, src_mask)
    output = self.decoder(output)
    return output